In [20]:
import os
import pickle
import numpy as np
import cupy as cp
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.signal as scisig
import scipy.stats
from collections import Counter
import scipy.interpolate as interp
import cupyx.scipy.signal as cusig
import neurokit2 as nk
from scipy.signal import find_peaks, welch

In [21]:
import faulthandler
faulthandler.enable()

In [22]:
fs_dict = {'ACC': 700, 'ECG': 700, 'EMG': 700, 'EDA': 700, 'Temp': 700, 'Resp': 700, 'label': 700}
WINDOW_IN_SECONDS = 30
STRIDE_IN_SECONDS = 0.75
label_dict = {'baseline': 1, 'stress': 2, 'amusement': 3}
int_to_label = {1: 'baseline', 2: 'stress', 3: 'amusement'}
feat_names = None
DATA_PATH = r'C:\Users\IALAB\Downloads\WESAD_TEST\data\WESAD/'
SAVE_PATH = r'C:\Users\IALAB\Downloads\WESAD_TEST\data_Complete_30_075/'


In [23]:
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

In [24]:
#def eda_stats(y):
#    Fs = fs_dict['EDA']
#    yn = (y - y.mean()) / y.std()
#    print(yn)
#    print("calculating eda stats")
#    [r, p, t, l, d, e, obj] = cvxEDA.cvxEDA(yn, 1. / Fs)
#    return [r, p, t, l, d, e, obj]

In [25]:
def compute_eda_metrics(eda_signal):
    eda_signal = eda_signal.flatten()
    scl_mean = np.mean(eda_signal)
    scl_std = np.std(eda_signal)

    diff_signal = np.diff(eda_signal)

    diff_signal = diff_signal.get() if hasattr(diff_signal, "get") else diff_signal  # Asegurar que es un NumPy array
    scr_peaks, _ = scisig.find_peaks(diff_signal, height=np.mean(diff_signal) + np.std(diff_signal))

    scr_mean = np.mean(diff_signal[scr_peaks]) if len(scr_peaks) > 0 else 0
    scr_std = np.std(diff_signal[scr_peaks]) if len(scr_peaks) > 0 else 0
    scr_count = len(scr_peaks)
    scr_amp = np.max(diff_signal[scr_peaks]) if scr_count > 0 else 0
    scr_sum = np.sum(diff_signal[scr_peaks])
    scr_area = np.trapz(diff_signal[scr_peaks])

    # Correlación de SCL con el tiempo
    time = np.arange(len(eda_signal))
    corr_SCL_t = np.corrcoef(time, eda_signal)[0, 1] if len(eda_signal) > 1 else 0

    return scl_mean, scl_std, scr_mean, scr_std, corr_SCL_t, scr_count, scr_amp, scr_sum, scr_area


In [26]:
def compute_emg_features(emg_signal, fs=fs_dict['EMG']):
    emg_signal = emg_signal.flatten()
    fxx, pxx = scisig.welch(emg_signal, fs=fs, nperseg=1024)
    
    # Frecuencia de pico
    f_peak = fxx[np.argmax(pxx)]
    
    # Bandas de frecuencia de interés
    psd_bands = {
        '0-10Hz': np.trapz(pxx[(fxx >= 0) & (fxx < 10)]),
        '10-20Hz': np.trapz(pxx[(fxx >= 10) & (fxx < 20)]),
        '20-50Hz': np.trapz(pxx[(fxx >= 20) & (fxx < 50)]),
        '50-100Hz': np.trapz(pxx[(fxx >= 50) & (fxx < 100)]),
        '100-150Hz': np.trapz(pxx[(fxx >= 100) & (fxx < 150)]),
        '150-250Hz': np.trapz(pxx[(fxx >= 150) & (fxx < 250)]),
        '250-500Hz': np.trapz(pxx[(fxx >= 250) & (fxx < 500)])
    }
    
    psd_total = sum(psd_bands.values())

    # Características de amplitud
    peak_indices, _ = scisig.find_peaks(emg_signal, height=np.mean(emg_signal) + np.std(emg_signal))
    peak_amp_values = emg_signal[peak_indices] if len(peak_indices) > 0 else [0]
    peak_count = len(peak_indices)
    peak_amp_mean = np.mean(peak_amp_values)
    peak_amp_std = np.std(peak_amp_values)
    peak_amp_sum = np.sum(peak_amp_values)
    peak_amp_norm = peak_amp_sum / len(emg_signal)
    
    return f_peak, psd_total, peak_count, peak_amp_mean, peak_amp_std, peak_amp_sum, peak_amp_norm

In [27]:
def compute_hrv_metrics(ecg_signal, fs = fs_dict['ECG']):
    ecg_signal = ecg_signal.flatten()
    peaks, _ = scisig.find_peaks(ecg_signal, height=np.mean(ecg_signal) + np.std(ecg_signal))

    if len(peaks) < 2:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

    rr_intervals = np.diff(peaks) * (1000 / fs)
    nn50 = np.sum(np.abs(np.diff(rr_intervals)) > 50)
    pnn50 = (nn50 / len(rr_intervals)) * 100
    rms_hrv = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    fxx, pxx = scisig.welch(rr_intervals, fs=4.0, nperseg=len(rr_intervals))
    lf_band = (0.04, 0.15)
    hf_band = (0.15, 0.4)

    lf = np.trapz(pxx[(fxx >= lf_band[0]) & (fxx <= lf_band[1])], fxx[(fxx >= lf_band[0]) & (fxx <= lf_band[1])])
    hf = np.trapz(pxx[(fxx >= hf_band[0]) & (fxx <= hf_band[1])], fxx[(fxx >= hf_band[0]) & (fxx <= hf_band[1])])
    lf_hf_ratio = lf / hf if hf > 0 else 0

    sum_f = np.sum(pxx)
    rel_f = sum_f / np.sum(rr_intervals) if np.sum(rr_intervals) > 0 else 0
    lf_norm = (lf / (lf + hf)) * 100 if (lf + hf) > 0 else 0
    hf_norm = (hf / (lf + hf)) * 100 if (lf + hf) > 0 else 0

    hist, bin_edges = np.histogram(rr_intervals, bins=50)
    tinn = np.max(hist) / np.mean(np.diff(bin_edges)) if np.mean(np.diff(bin_edges)) > 0 else 0

    return nn50, pnn50, tinn, rms_hrv, lf, hf, lf_hf_ratio, sum_f, rel_f, lf_norm, hf_norm

In [28]:
def compute_emg_peaks(emg_signal, threshold=0.05):
    # Normalizar señal EMG
    emg_signal = emg_signal.flatten()
    emg_signal = (emg_signal - np.min(emg_signal)) / (np.max(emg_signal) - np.min(emg_signal))

    # Detectar picos que superen el umbral
    peaks, _ = scisig.find_peaks(emg_signal, height=threshold)
    
    # Obtener amplitudes de los picos detectados
    peak_amplitudes = emg_signal[peaks] if len(peaks) > 0 else [0]

    return len(peaks), np.mean(peak_amplitudes)

In [29]:
def compute_respiration_metrics(resp_signal, fs=fs_dict['Resp']):
    resp_signal = resp_signal.flatten()
    peaks, _ = scisig.find_peaks(resp_signal, height=np.mean(resp_signal) + np.std(resp_signal))
    troughs, _ = scisig.find_peaks(-resp_signal, height=-np.mean(resp_signal) - np.std(resp_signal))

    if len(peaks) < 2 or len(troughs) < 2:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0

    inspiration_durations = np.diff(peaks) / fs
    expiration_durations = np.diff(troughs) / fs

    I_mean = np.mean(inspiration_durations)
    I_std = np.std(inspiration_durations)
    E_mean = np.mean(expiration_durations)
    E_std = np.std(expiration_durations)

    ie_ratio = I_mean / E_mean if E_mean > 0 else 0
    resp_range = np.max(resp_signal) - np.min(resp_signal)
    insp_vol = np.mean(resp_signal[peaks]) - np.mean(resp_signal[troughs])
    resp_rate = len(peaks) / (len(resp_signal) / fs)
    resp_duration = len(resp_signal) / fs

    return I_mean, I_std, E_mean, E_std, ie_ratio, resp_range, insp_vol, resp_rate, resp_duration


In [30]:
class SubjectData:

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject_keys = ['signal', 'label', 'subject']
        self.signal_keys = ['chest', 'wrist']
        self.chest_keys = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'Resp': self.data['signal']['chest']['Resp']})
        return data

    def get_chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.get_chest_data()[key].flatten(), self.labels, key)
                for key in self.chest_keys
            }
        return results

In [31]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scisig.butter(order, normal_cutoff, btype='low', analog=False)  # Usa scipy.signal aquí
    return cp.array(b), cp.array(a)  # Convierte los coeficientes a cupy si es necesario


def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order)
    y = cp.asnumpy(data)  # Convierte los datos de cupy a numpy antes de filtrar
    y = scisig.lfilter(cp.asnumpy(b), cp.asnumpy(a), y)  # Aplica el filtro en numpy
    return cp.array(y)  # Convierte el resultado de nuevo a cupy si es necesario

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs  # Frecuencia de Nyquist
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def get_slope(signal):
    resu = (signal[-1] - signal[0]) / len(signal)
    resu = resu[0] if isinstance(resu, cp.ndarray) else resu
    return resu

def get_peak_freq(x, fs):
    f, Pxx = scisig.periodogram(x, fs=fs)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    return psd_dict[max(psd_dict.keys())]

def get_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features


def get_net_accel(data):
    return (data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2).apply(lambda x: np.sqrt(x))


def get_peak_freq(x):
    f, Pxx = scisig.periodogram(x, fs=8)
    psd_dict = {amp: freq for amp, freq in zip(Pxx, f)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq


# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
def filterSignalFIR(eda, cutoff=0.4, numtaps=64):
    f = cutoff / (fs_dict['ACC'] / 2.0)
    FIR_coeff = scisig.firwin(numtaps, f)

    return scisig.lfilter(FIR_coeff, 1, eda.flatten())

def compute_hr(ecg_signal, fs):
    # Filtro pasa bandas para eliminar ruido
    ecg_filtered = butter_bandpass_filter(ecg_signal, lowcut=0.5, highcut=40, fs=fs, order=4)

    # Detectar picos R
    peaks, _ = find_peaks(ecg_filtered, distance=fs*0.6, height=np.mean(ecg_filtered) + np.std(ecg_filtered))

    # Calcular intervalos RR (diferencias entre picos)
    rr_intervals = np.diff(peaks) / fs  # Convertir a segundos

    # Calcular HR (latidos por minuto)
    if len(rr_intervals) > 0:
        hr_mean = 60 / np.mean(rr_intervals)
        hr_std = np.std(60 / rr_intervals)  # Variabilidad de HR
    else:
        hr_mean, hr_std = np.nan, np.nan  # En caso de que no se detecten picos

    return hr_mean, hr_std

def triangle(x, a, b, c):
    return np.maximum(0, a - np.abs(x - b) / c)

In [32]:
def compute_features(data_dict, fs_dict, norm_type=None):
    feature_dict = {}

    # ECG y BVP
    ecg_signal = data_dict['ECG']
    hr_mean, hr_std = compute_hr(ecg_signal, fs_dict['ECG'])
    feature_dict['HR_mean'] = hr_mean
    feature_dict['HR_std'] = hr_std
    
    nn50, pNN50, tinn, rmsHRV, lf, hf, lf_hf, sum_f, rel_f, lf_norm, hf_norm = compute_hrv_metrics(ecg_signal)
    feature_dict.update({
        'NN50': nn50, 'pNN50': pNN50, 'TINN': tinn, 
        'rmsHRV': rmsHRV, 'LF': lf, 'HF': hf, 'LF_HF': lf_hf,
        'sum_f': sum_f, 'rel_f': rel_f, 'LF_norm': lf_norm, 'HF_norm': hf_norm
    })

    # EDA
    eda_signal = butter_lowpass_filter(data_dict['EDA'], 5.0, fs_dict['EDA'], 6)
    feature_dict.update({
        'EDA_mean': np.mean(eda_signal), 'EDA_std': np.std(eda_signal),
        'EDA_min': np.min(eda_signal), 'EDA_max': np.max(eda_signal),
        'EDA_range': np.max(eda_signal) - np.min(eda_signal), 'EDA_slope': get_slope(eda_signal)
    })
    scl_mean, scl_std, scr_mean, scr_std, corr_scl_t, scr_count, scr_amp, scr_sum, scr_area = compute_eda_metrics(eda_signal)

    feature_dict.update({
        'scl_mean': scl_mean, 'scl_std': scl_std, 'scr_mean': scr_mean, 'scr_std': scr_std,
        'corr_scl_t': corr_scl_t, 'scr_count': scr_count, 'scr_amp': scr_amp,
        'scr_sum': scr_sum, 'scr_area': scr_area
    })

    # EMG
    emg_signal = data_dict['EMG']
    feature_dict.update({
        'EMG_mean': np.mean(emg_signal), 'EMG_std': np.std(emg_signal),
        'EMG_median': np.median(emg_signal), 'EMG_p10': np.percentile(emg_signal, 10),
        'EMG_p90': np.percentile(emg_signal, 90), 'EMG_range': np.max(emg_signal) - np.min(emg_signal),
        'EMG_sum': np.sum(np.abs(emg_signal))
    })
    f_peak, psd_bands, peak_count, peak_amp_mean, peak_amp_std, peak_amp_sum, peak_amp_norm = compute_emg_features(emg_signal, fs_dict['EMG'])
    feature_dict.update({
        'EMG_f_peak': f_peak, 'EMG_PSD_bands': psd_bands,
        'EMG_peak_count': peak_count, 'EMG_peak_amp_mean': peak_amp_mean,
        'EMG_peak_amp_std': peak_amp_std, 'EMG_peak_amp_sum': peak_amp_sum,
        'EMG_peak_amp_norm': peak_amp_norm
    })

    # Respiración
    resp_signal = data_dict['Resp']
    feature_dict.update({
        'Resp_mean': np.mean(resp_signal), 'Resp_std': np.std(resp_signal),
    })
    I_mean, I_std, E_mean, E_std, ie_ratio, resp_range, insp_vol, resp_rate, resp_duration = compute_respiration_metrics(resp_signal, fs_dict['Resp'])
    feature_dict.update({
        'Resp_I_mean': I_mean, 'Resp_I_std': I_std, 'Resp_E_mean': E_mean, 'Resp_E_std': E_std,
        'Resp_IE_ratio': ie_ratio, 'Resp_range': resp_range, 'Resp_insp_vol': insp_vol,
        'Resp_rate': resp_rate, 'Resp_duration': resp_duration
    })

    # Temperatura
    temp_signal = data_dict['Temp']
    feature_dict.update({
        'Temp_mean': np.mean(temp_signal), 'Temp_std': np.std(temp_signal),
        'Temp_min': np.min(temp_signal), 'Temp_max': np.max(temp_signal),
        'Temp_range': np.max(temp_signal) - np.min(temp_signal), 'Temp_slope': get_slope(temp_signal)
    })

    # Convertir a DataFrame con solo una fila
    df = pd.DataFrame([feature_dict])

    df["EDA_slope"] = df["EDA_slope"].apply(lambda x: x[0] if isinstance(x, list) else x)
    df["Temp_slope"] = df["Temp_slope"].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else (x[0] if isinstance(x, list) else x))


    # Normalización opcional
    if norm_type == 'std':
        df = (df - df.mean()) / df.std()
    elif norm_type == 'minmax':
        df = (df - df.min()) / (df.max() - df.min())

    return df

In [33]:
def get_samples(data_dict, labels, fs_dict, stride_seconds):
    global feat_names
    global WINDOW_IN_SECONDS

    samples = []
    all_samples = []
    
    # Convertir tiempo a muestras
    window_len = int(fs_dict['ECG'] * WINDOW_IN_SECONDS)  # Se toma una señal como referencia
    stride_len = int(fs_dict['ECG'] * stride_seconds)  

    num_ventanas = (len(labels) - window_len) // stride_len + 1
    print(f"El número de ventanas esperadas es: {num_ventanas}")

    last_progress = -10
    processed = 0

    all_samples = pd.DataFrame()

    for start in range(0, len(labels) - window_len + 1, stride_len):

        processed += 1  # Incrementar contador manualmente
        progress = processed / num_ventanas * 100

        if processed % 50 == 0:

            print(f"\rProgreso: {progress:.4f}% completado", end="", flush=True)
        
        last_progress = progress
        # Extraer ventana de cada señal
        window_data = {key: val[start:start + window_len] for key, val in data_dict.items()}
        window_labels = labels[start:start + window_len]  # Extraer etiquetas de la ventana

        # Aplicar hard labeling: etiqueta más frecuente en la ventana
        label_counts = Counter(window_labels)
        most_common_labels = label_counts.most_common()  # [(label1, count1), (label2, count2), ...]
        
        # Si hay empate, tomar la primera que aparece en la ventana original
        max_count = most_common_labels[0][1]
        candidate_labels = [label for label, count in most_common_labels if count == max_count]
        chosen_label = next(label for label in window_labels if label in candidate_labels)

        # Calcular características con compute_features

        features_df = compute_features(window_data, fs_dict)

        # Agregar la etiqueta al DataFrame de features
        features_df['label'] = chosen_label

        all_samples = pd.concat([all_samples, features_df], ignore_index=True)

    if all_samples.empty:
        print("Advertencia: No se generaron muestras en get_samples(), devolviendo DataFrame vacío.")

    print("\n Procesamiento de ventanas completado.")  
    
    return all_samples

In [34]:
def make_patient_data(subject_id):
    global SAVE_PATH
    global WINDOW_IN_SECONDS

    # Make subject data object for Sx
    subject = SubjectData(main_path=DATA_PATH, subject_number=subject_id)

    # Empatica E4 data - now with resp
    data_dict = subject.get_chest_data()

    print(data_dict.keys())
    print(subject.labels)

    # norm type
    norm_type = None

    # The 3 classes we are classifying
    grouped = subject.labels
    print(len(grouped))
    baseline = grouped[grouped == 1]
    print(len(baseline))
    stress = grouped[grouped == 2]
    print(len(stress))
    amusement = grouped[grouped == 3]
    print(len(amusement))

    total_data = len(baseline) + len(stress) + len(amusement)
    print("DATA A TRABAJAR: " + str(total_data))
    
    # print(f'Available windows for {subject.name}:')
    samples_per_window = int(fs_dict['label'] * WINDOW_IN_SECONDS)
    stride_per_window = int(fs_dict['label'] * STRIDE_IN_SECONDS)

    n_wdws = (total_data - samples_per_window) / stride_per_window + 1
    # print(f'Baseline: {n_baseline_wdws}\nStress: {n_stress_wdws}\nAmusement: {n_amusement_wdws}\n')
    print(f"Procesando S{subject_id}:")
    print(f"  - Windows: {n_wdws}")


    valid_labels = {1, 2, 3}
    mask = np.isin(subject.labels, list(valid_labels))
    filtered_labels = subject.labels[mask]
    filtered_signals = {}
    for key in data_dict.keys():
        filtered_signals[key] = data_dict[key][mask] 

    print("Data lista para procesar: " + str(len(filtered_labels)))

    samples = get_samples(filtered_signals, filtered_labels, fs_dict=fs_dict, stride_seconds = STRIDE_IN_SECONDS)

    print("features calculated")

    if not isinstance(samples, pd.DataFrame):
        all_samples = pd.concat(samples, ignore_index=True)
    else:
        all_samples = samples.copy()

    all_samples['label'] = all_samples['label'].astype(int)
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1)

    # Guarda el archivo CSV
    all_samples.to_csv(f'{SAVE_PATH}/S{subject_id}_feats_4.csv', index=False)

    # Liberar memoria
    subject = None
    all_samples = None
    samples = None

In [35]:
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{SAVE_PATH}/S{s}_feats_4.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    print(df.head(10))
    print(df.columns)

    df['label'] = df[['1', '2', '3']].idxmax(axis=1).astype(int)
    df.drop(['1', '2', '3'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)

    df.to_csv(f'{SAVE_PATH}/may14_feats4.csv')

    counts = df['label'].value_counts()

    print("Índices en counts:", counts.index.tolist())
    print("Claves en int_to_label:", int_to_label.keys())

    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_to_label[label]}: {number}')

In [36]:
#subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

for patient in subject_ids:
    print(f'Processing data for S{patient}...')
    make_patient_data(patient)


Processing data for S2...
dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])
[0 0 0 ... 0 0 0]
4255300
800800
430500
253400
DATA A TRABAJAR: 1484700
Procesando S2:
  - Windows: 2789.0
Data lista para procesar: 1484700
El número de ventanas esperadas es: 2789
Progreso: 98.6016% completado
 Procesamiento de ventanas completado.
features calculated
Processing data for S3...
dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])
[0 0 0 ... 0 0 0]
4545100
798000
448000
262500
DATA A TRABAJAR: 1508500
Procesando S3:
  - Windows: 2834.3333333333335
Data lista para procesar: 1508500
El número de ventanas esperadas es: 2834
Progreso: 98.8003% completado
 Procesamiento de ventanas completado.
features calculated
Processing data for S4...
dict_keys(['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'])
[0 0 0 ... 0 0 0]
4496100
810601
444500
260400
DATA A TRABAJAR: 1515501
Procesando S4:
  - Windows: 2847.6685714285713
Data lista para procesar: 1515501
El número de ventanas esperadas es: 2847
Progre

In [37]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

combine_files(subject_ids)
print('Processing complete.')

             HR_std  NN50      pNN50      TINN      rmsHRV           LF  \
HR_mean                                                                   
 0.001771  0.131424   134  63.207547  7.440476  346.208974  1487.321479   
 0.001239  0.130833   136  62.385321  7.619048  333.785207  1705.577825   
 0.000758  0.130931   138  62.443439  7.678571  324.676077  1581.800248   
-0.000221  0.133575   137  63.720930  7.500000  328.120693  1358.006821   
 0.001012  0.133966   137  64.319249  7.440476  344.163462  1509.047401   
 0.001831  0.134757   136  61.261261  7.678571  331.091918  1465.974838   
 0.000110  0.134100   137  60.619469  7.976190  327.413931  1264.384218   
 0.000575  0.133923   137  60.619469  7.976190  326.508666  1202.831543   
-0.000750  0.134468   136  60.714286  7.857143  323.469950  1071.756047   
 0.000264  0.134938   137  60.888889  7.916667  328.257878  1062.709476   

                    HF     LF_HF         sum_f      rel_f  ...  Temp_mean  \
HR_mean               

In [38]:
print('holi')

holi
